<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lesson-10:-Problem-Set-5:-MapReduce-on-Subway-Data" data-toc-modified-id="Lesson-10:-Problem-Set-5:-MapReduce-on-Subway-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lesson 10: Problem Set 5: MapReduce on Subway Data</a></span><ul class="toc-item"><li><span><a href="#Lesson-10:-Quiz-1:-Ridership-per-Station" data-toc-modified-id="Lesson-10:-Quiz-1:-Ridership-per-Station-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Lesson 10: Quiz 1: Ridership per Station</a></span><ul class="toc-item"><li><span><a href="#Pandas-method" data-toc-modified-id="Pandas-method-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Pandas method</a></span></li></ul></li><li><span><a href="#Lesson-10:-Quiz-2:-Ridership-by-Weather-Type" data-toc-modified-id="Lesson-10:-Quiz-2:-Ridership-by-Weather-Type-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Lesson 10: Quiz 2: Ridership by Weather Type</a></span><ul class="toc-item"><li><span><a href="#Pandas-method" data-toc-modified-id="Pandas-method-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Pandas method</a></span></li></ul></li><li><span><a href="#Lesson-10:-Quiz-3:-Busiest-Hour" data-toc-modified-id="Lesson-10:-Quiz-3:-Busiest-Hour-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Lesson 10: Quiz 3: Busiest Hour</a></span><ul class="toc-item"><li><span><a href="#Pandas-method" data-toc-modified-id="Pandas-method-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Pandas method</a></span></li></ul></li></ul></li></ul></div>

# Lesson 10: Problem Set 5: MapReduce on Subway Data

Data: https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv

In [1]:
!head -n2 ../data/turnstile_data_master_with_weather.csv

,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,R001,2011-05-01,01:00:00,1,REGULAR,0.0,0.0,30.31,42.0,35.0,30.23,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0


From this data, we can see data[0] has no column name, data[1] is UNIT and data[6] is ENTRIESn_hourly. We will print this unit and hourly entry separated by tab for the mapper code. The mapper.py will take data from bash command `cat data.csv` and emits unit and hourly entries separated by tab.

Then we sort this data and send it to the reducer.

In [2]:
import pandas as pd

filename = '../data/turnstile_data_master_with_weather.csv'
df = pd.read_csv(filename, parse_dates = True)
hdr = df.columns.values
for i,v in enumerate(hdr):
    print(i,v)

0 Unnamed: 0
1 UNIT
2 DATEn
3 TIMEn
4 Hour
5 DESCn
6 ENTRIESn_hourly
7 EXITSn_hourly
8 maxpressurei
9 maxdewpti
10 mindewpti
11 minpressurei
12 meandewpti
13 meanpressurei
14 fog
15 rain
16 meanwindspdi
17 mintempi
18 meantempi
19 maxtempi
20 precipi
21 thunder


## Lesson 10: Quiz 1: Ridership per Station

In [3]:
%%writefile lesson10_quiz1_ridership_per_station_mapper.py
import sys
import string
import logging

# from util import mapper_logfile
# logging.basicConfig(filename=mapper_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')

def mapper():
    """
    The input to this mapper will be the final Subway-MTA dataset, the same as
    in the previous exercise.  You can check out the csv and its structure below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv

    For each line of input, the mapper output should PRINT (not return) the UNIT as 
    the key, the number of ENTRIESn_hourly as the value, and separate the key and 
    the value by a tab. For example: 'R002\t105105.0'

    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    
    The logging module can be used to give you more control over your debugging
    or other messages than you can get by printing them. In this exercise, print
    statements from your mapper will go to your reducer, and print statements
    from your reducer will be considered your final output. By contrast, messages
    logged via the loggers we configured will be saved to two files, one
    for the mapper and one for the reducer. If you click "Test Run", then we
    will show the contents of those files once your program has finished running.
    The logging module also has other capabilities; see 
    https://docs.python.org/2/library/logging.html for more information.
    """

    for line in sys.stdin:
        data = line.strip().split(",")
        if data[1] == "UNIT":
            continue

        # print UNIT and ENTRIESn_hourly separated by tab from turn-wtr data
        print ("{}\t{}".format(data[1],data[6]))

mapper()

Overwriting lesson10_quiz1_ridership_per_station_mapper.py


In [4]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz1_ridership_per_station_mapper.py > ridership_mapper.csv;\
head -n5 ridership_mapper.csv

R001	0.0
R001	217.0
R001	890.0
R001	2451.0
R001	4400.0


In [5]:
%%writefile lesson10_quiz1_ridership_per_station_reducer.py
import sys
import logging

# from util import reducer_logfile
# logging.basicConfig(filename=reducer_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')

def reducer():
    '''
    Given the output of the mapper for this exercise, the reducer should PRINT 
    (not return) one line per UNIT along with the total number of ENTRIESn_hourly 
    over the course of May (which is the duration of our data), separated by a tab.
    An example output row from the reducer might look like this: 'R001\t500625.0'

    You can assume that the input to the reducer is sorted such that all rows
    corresponding to a particular UNIT are grouped together.

    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    '''
    counter = {}
    for line in sys.stdin:
        # your code here
        data = line.strip().split("\t")
        if data[0] in counter:
            counter[data[0]] += int(float(data[1]))
        else:
            counter[data[0]] = int(float(data[1]))

    for i in counter: 
        print (i, "\t", counter[i])

reducer()

Overwriting lesson10_quiz1_ridership_per_station_reducer.py


In [6]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz1_ridership_per_station_mapper.py |\
sort |\
python lesson10_quiz1_ridership_per_station_reducer.py | head -n5

R001 	 749682
R002 	 176535
R003 	 35938
R004 	 93104
R005 	 91031


### Pandas method

In [7]:
import pandas as pd
filename = '../data/turnstile_data_master_with_weather.csv'
df = pd.read_csv(filename, parse_dates = True)
df.head()

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,0,R001,2011-05-01,01:00:00,1,REGULAR,0.0,0.0,30.31,42.0,...,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0
1,1,R001,2011-05-01,05:00:00,5,REGULAR,217.0,553.0,30.31,42.0,...,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0
2,2,R001,2011-05-01,09:00:00,9,REGULAR,890.0,1262.0,30.31,42.0,...,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0
3,3,R001,2011-05-01,13:00:00,13,REGULAR,2451.0,3708.0,30.31,42.0,...,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0
4,4,R001,2011-05-01,17:00:00,17,REGULAR,4400.0,2501.0,30.31,42.0,...,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0


In [8]:
df = df.groupby('UNIT').sum()
df.head()['ENTRIESn_hourly']

UNIT
R001    749682.0
R002    176535.0
R003     35938.0
R004     93104.0
R005     91031.0
Name: ENTRIESn_hourly, dtype: float64

In [9]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz1_ridership_per_station_mapper.py | sort |\
python lesson10_quiz1_ridership_per_station_reducer.py | head -n 5

R001 	 749682
R002 	 176535
R003 	 35938
R004 	 93104
R005 	 91031


## Lesson 10: Quiz 2: Ridership by Weather Type

In [10]:
%%writefile lesson10_quiz2_ridership_by_weather_mapper.py
import sys
import string
import logging

# from util import mapper_logfile
# logging.basicConfig(filename=mapper_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')


HEADER_STRING = ",UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder"
HEADERS = HEADER_STRING.split(",")
FOG_INDEX = HEADERS.index('fog')
RAIN_INDEX = HEADERS.index('rain')
ENTRIESN_HOURLY_INDEX = HEADERS.index('ENTRIESn_hourly')


def mapper():
    '''
    For this exercise, compute the average value of the ENTRIESn_hourly column 
    for different weather types. Weather type will be defined based on the 
    combination of the columns fog and rain (which are boolean values).
    For example, one output of our reducer would be the average hourly entries 
    across all hours when it was raining but not foggy.

    Each line of input will be a row from our final Subway-MTA dataset in csv format.
    You can check out the input csv file and its structure below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv
    
    Note that this is a comma-separated file.

    This mapper should PRINT (not return) the weather type as the key (use the 
    given helper function to format the weather type correctly) and the number in 
    the ENTRIESn_hourly column as the value. They should be separated by a tab.
    For example: 'fog-norain\t12345'
    
    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    '''

    # Takes in variables indicating whether it is foggy and/or rainy and
    # returns a formatted key that you should output.  The variables passed in
    # can be booleans, ints (0 for false and 1 for true) or floats (0.0 for
    # false and 1.0 for true), but the strings '0.0' and '1.0' will not work,
    # so make sure you convert these values to an appropriate type before
    # calling the function.
    def format_key(fog, rain):
        return '{}fog-{}rain'.format(
            '' if fog else 'no',
            '' if rain else 'no'
        )

    for line in sys.stdin:
        if "UNIT,DATEn" in line:
            continue
        data = line.split(',')
        fog = int(float(data[FOG_INDEX]))
        rain = int(float(data[RAIN_INDEX]))
        entriesn_hourly = data[ENTRIESN_HOURLY_INDEX]

        key = format_key(fog, rain)

        print ("{0}\t{1}".format(key, entriesn_hourly))

mapper()

Overwriting lesson10_quiz2_ridership_by_weather_mapper.py


In [11]:
!head -n2 ../data/turnstile_data_master_with_weather.csv

,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,R001,2011-05-01,01:00:00,1,REGULAR,0.0,0.0,30.31,42.0,35.0,30.23,39.0,30.27,0.0,0.0,5.0,50.0,60.0,69.0,0.0,0.0


In [12]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz2_ridership_by_weather_mapper.py > weather_mapper.csv ;\
head -n 5 weather_mapper.csv

nofog-norain	0.0
nofog-norain	217.0
nofog-norain	890.0
nofog-norain	2451.0
nofog-norain	4400.0


In [13]:
%%writefile lesson10_quiz2_ridership_by_weather_reducer.py
import sys
import logging

# from util import reducer_logfile
# logging.basicConfig(filename=reducer_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')




def reducer():
    '''
    Given the output of the mapper for this assignment, the reducer should
    print one row per weather type, along with the average value of
    ENTRIESn_hourly for that weather type, separated by a tab. You can assume
    that the input to the reducer will be sorted by weather type, such that all
    entries corresponding to a given weather type will be grouped together.

    In order to compute the average value of ENTRIESn_hourly, you'll need to
    keep track of both the total riders per weather type and the number of
    hours with that weather type. That's why we've initialized the variable 
    riders and num_hours below. Feel free to use a different data structure in 
    your solution, though.

    An example output row might look like this:
    'fog-norain\t1105.32467557'

    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    '''

    riders = 0      # The number of total riders for this key
    num_hours = 0   # The number of hours with this key
    old_key = None

    for line in sys.stdin:
        # your code here

        data = line.strip().split("\t")
        
        if len(data) != 2:
            continue
        
        this_key, count = data
        
        if old_key and old_key != this_key:
            print ("{0}\t{1}".format(old_key, num_hours/riders))
            num_hours = 0
            riders = 0
            
        old_key = this_key
        num_hours += float(count)
        riders += 1
        
    if old_key != None:
        print ("{0}\t{1}".format(old_key, num_hours/riders))

reducer()


Overwriting lesson10_quiz2_ridership_by_weather_reducer.py


In [14]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz2_ridership_by_weather_mapper.py |\
sort |\
python lesson10_quiz2_ridership_by_weather_reducer.py

fog-norain	1315.5798068077277
fog-rain	1115.1315179933338
nofog-norain	1078.5467969676283
nofog-rain	1098.9533007612772


### Pandas method

In [15]:
import pandas as pd
filename = '../data/turnstile_data_master_with_weather.csv'
df = pd.read_csv(filename, parse_dates = True)
df.head();

In [16]:
df = df.groupby(['fog','rain']).mean()
df.head()['ENTRIESn_hourly']

fog  rain
0.0  0.0     1078.546797
     1.0     1098.953301
1.0  0.0     1315.579807
     1.0     1115.131518
Name: ENTRIESn_hourly, dtype: float64

## Lesson 10: Quiz 3: Busiest Hour

In [17]:
%%writefile lesson10_quiz3_busiest_hour_mapper.py
import sys
import string
import logging

# from util import mapper_logfile
# logging.basicConfig(filename=mapper_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')


HEADER_STRING = ",UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder"
HEADERS = HEADER_STRING.split(",")
ENTRIESN_HOURLY_INDEX = HEADERS.index('ENTRIESn_hourly')
UNIT_INDEX = HEADERS.index('UNIT')
DATEN_INDEX = HEADERS.index('DATEn')
TIMEN_INDEX = HEADERS.index('TIMEn')


def mapper():
    """
    In this exercise, for each turnstile unit, you will determine the date and time 
    (in the span of this data set) at which the most people entered through the unit.
    
    The input to the mapper will be the final Subway-MTA dataset, the same as
    in the previous exercise. You can check out the csv and its structure below:
    https://s3.amazonaws.com/content.udacity-data.com/courses/ud359/turnstile_data_master_with_weather.csv

    For each line, the mapper should return the UNIT, ENTRIESn_hourly, DATEn, and 
    TIMEn columns, separated by tabs. For example:
    'R001\t100000.0\t2011-05-01\t01:00:00'

    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    """

    for line in sys.stdin:
        if "UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly" in line:
            continue

        data = line.split(",")
        unit = data[UNIT_INDEX]
        date_n = data[DATEN_INDEX]
        time_n = data[TIMEN_INDEX]
        entriesn_hourly = float(data[ENTRIESN_HOURLY_INDEX])

        print ("{0}\t{1}\t{2}\t{3}".format(unit, entriesn_hourly, date_n, time_n))


mapper()



Overwriting lesson10_quiz3_busiest_hour_mapper.py


In [18]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz3_busiest_hour_mapper.py > busiest_hour_mapper.csv;\
head -n5 busiest_hour_mapper.csv

R001	0.0	2011-05-01	01:00:00
R001	217.0	2011-05-01	05:00:00
R001	890.0	2011-05-01	09:00:00
R001	2451.0	2011-05-01	13:00:00
R001	4400.0	2011-05-01	17:00:00


In [19]:
%%writefile lesson10_quiz3_busiest_hour_reducer.py
import sys
import logging

# from util import reducer_logfile
# logging.basicConfig(filename=reducer_logfile, format='%(message)s',
#                     level=logging.INFO, filemode='w')

def reducer():
    '''
    Write a reducer that will compute the busiest date and time (that is, the 
    date and time with the most entries) for each turnstile unit. Ties should 
    be broken in favor of datetimes that are later on in the month of May. You 
    may assume that the contents of the reducer will be sorted so that all entries 
    corresponding to a given UNIT will be grouped together.
    
    The reducer should print its output with the UNIT name, the datetime (which 
    is the DATEn followed by the TIMEn column, separated by a single space), and 
    the number of entries at this datetime, separated by tabs.

    For example, the output of the reducer should look like this:
    R001    2011-05-11 17:00:00	   31213.0
    R002	2011-05-12 21:00:00	   4295.0
    R003	2011-05-05 12:00:00	   995.0
    R004	2011-05-12 12:00:00	   2318.0
    R005	2011-05-10 12:00:00	   2705.0
    R006	2011-05-25 12:00:00	   2784.0
    R007	2011-05-10 12:00:00	   1763.0
    R008	2011-05-12 12:00:00	   1724.0
    R009	2011-05-05 12:00:00	   1230.0
    R010	2011-05-09 18:00:00	   30916.0
    ...
    ...

    Since you are printing the output of your program, printing a debug 
    statement will interfere with the operation of the grader. Instead, 
    use the logging module, which we've configured to log to a file printed 
    when you click "Test Run". For example:
    logging.info("My debugging message")
    Note that, unlike print, logging.info will take only a single argument.
    So logging.info("my message") will work, but logging.info("my","message") will not.
    '''

    max_entries = 0
    old_key = None
    datetime = ''

    for line in sys.stdin:
        # your code here
        data = line.strip().split("\t")
        
        if len(data) != 4:
            continue
        
        this_key, count, date, time = data
        
        if old_key and old_key != this_key:
            print ("{0}\t{1}\t{2}".format(old_key, datetime, max_entries))
            max_entries = 0
            datetime = ''
        
        old_key = this_key
        if float(count) >= max_entries:
            max_entries = float(count)
            datetime = '{0} {1}'.format(date, time)
        elif max_entries == 0:
            datetime = '{0} {1}'.format(date, time)
            max_entries = 0.0

        
    if old_key != None:
        print ("{0}\t{1}\t{2}".format(old_key, datetime, max_entries))

reducer()

Overwriting lesson10_quiz3_busiest_hour_reducer.py


In [24]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz3_busiest_hour_mapper.py |\
sort |\
python lesson10_quiz3_busiest_hour_reducer.py | head -n 5

R001	2011-05-11 17:00:00	31213.0
R002	2011-05-12 21:00:00	4295.0
R003	2011-05-05 12:00:00	995.0
R004	2011-05-12 12:00:00	2318.0
R005	2011-05-10 12:00:00	2705.0


### Pandas method

In [21]:
import pandas as pd
filename = '../data/turnstile_data_master_with_weather.csv'
df = pd.read_csv(filename, parse_dates = True)
df.head();

In [22]:
df = df.groupby(['UNIT']).max()
df.head()[['DATEn','TIMEn','ENTRIESn_hourly']]

,DATEn,TIMEn,ENTRIESn_hourly
UNIT,,,
R001,2011-05-30,21:00:00,31213.0
R002,2011-05-30,21:00:00,4295.0
R003,2011-05-30,20:00:00,995.0
R004,2011-05-30,20:00:00,2318.0
R005,2011-05-30,20:00:00,2705.0


In [25]:
!cat ../data/turnstile_data_master_with_weather.csv |\
python lesson10_quiz3_busiest_hour_mapper.py |\
sort |\
python lesson10_quiz3_busiest_hour_reducer.py | head -n 5

R001	2011-05-11 17:00:00	31213.0
R002	2011-05-12 21:00:00	4295.0
R003	2011-05-05 12:00:00	995.0
R004	2011-05-12 12:00:00	2318.0
R005	2011-05-10 12:00:00	2705.0
